In [35]:
import os
import requests
import pandas

In [60]:
# Export folder - Be Careful Python overwrites existing data silently!:
target_path = os.path.join("~/Documents/", "data")
os.makedirs(target_path, exist_ok=True)

# Hashtags: for each of these one csv report is generated
relevant_hashtags_list = ["hotosm-project-*", "hotosm-project-15476"]

# Query Parameters:
startdate="2023-02-13T00:00:00Z"
enddate="2024-02-13T23:59:59Z"
iso3countryCodes  = []
topics = ["healthcare", "lulc"]


data = {
    "startdate": startdate,
    "enddate": enddate,
    "countries": ",".join(iso3countryCodes)
}

# big-numbers / summary section:
for hashtag in relevant_hashtags_list:
    df = pandas.DataFrame(requests.get(f"https://stats.now.ohsome.org/api/stats/{hashtag}", params=data).json())
    df = df.loc[ ["changesets","users", "roads", "buildings", "edits","latest"] , "result" ]
    df_topics = pandas.DataFrame(requests.get(f"https://stats.now.ohsome.org/api/topic/{','.join(topics)}", params= data | {"hashtag": hashtag} ).json()["result"])

    for column in df_topics.columns:
        df[column] = df_topics[column]["value"]
        
    df.to_csv(os.path.join(target_path, f"{hashtag}_{startdate}-{enddate}_{'-'.join(iso3countryCodes)}_{'-'.join(topics)}.csv"))